In [1]:
# importing necessary libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


# Processing-part-1: extracting raw table (from webpage)

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
#print(url)
#print(result.status_code)
#print(result.headers)

will get data + clean it


In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['M1C', 'Scarborough', 'Highland Creek'],
 ['M1C', 'Scarborough', 'Rouge Hill'],
 ['M1C', 'Scarborough', 'Port Union'],
 ['M3C', 'North 

convert into a dataframe


In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(lst, columns=cols)
print(df_toronto.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


custom groupby / agg to merge Neighborhoods
groupby PostalCode, keep the first Borough and join() Neighborhoods

In [5]:
df_toronto = df_toronto.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

will check if 'M5A' example is done correctly


In [6]:
df_toronto.loc[df_toronto['PostalCode'] == 'M5A']


,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


will check the df.shape


In [7]:
df_toronto.shape


(103, 3)

Part 1: Pre-processing - Installing geocoder

# Part 2: Processing - Extracting Latitude and Longtitude for each Postal code in Toronto

In [8]:
df_toronto.to_csv('Toronto1_df.csv',index=False)

In [9]:
Toronto1_df = "Toronto1_df.csv"
TorontoPostalCodes = pd.read_csv(Toronto1_df).set_index("PostalCode")
TorontoPostalCodes.rename_axis("Postal Code", axis='index', inplace=True)
TorontoPostalCodes.head()

,Borough,Neighborhood
Postal Code,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [10]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Combining Two data frames with mapped postcodes


In [11]:
toronto_neighborhoods = TorontoPostalCodes.join(geocsv_data)
toronto_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
toronto_neighborhoods.to_csv('Toronto_N_df.csv',index=False)

In [13]:

toronto_neighborhoods

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


Create a map of Toronto

In [14]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [15]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighborhoods['Borough'].unique()),
        toronto_neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


# 1.Scarborough and its neighbourhoods

In [17]:
scarborough_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
scarborough_data.shape

(17, 4)

In [19]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [20]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

In [21]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.806686299999996, -79.19435340000001.


In [22]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [23]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

And how many venues were returned by Foursquare?

In [26]:
print('{} venues were returned by Foursquare.'.format(SGnearby_venues.shape[0]))

10 venues were returned by Foursquare.


# 2.North York and its neighbourhoods

In [27]:
northyork_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,North York,Hillcrest Village,43.803762,-79.363452
1,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,North York,Bayview Village,43.786947,-79.385975
3,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [28]:
northyork_data.shape

(24, 4)

In [29]:
address_ny = 'North York,Toronto'
latitude_ny = 43.7615
longitude_ny = -79.4111
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_ny, longitude_ny))

The geograpical coordinate of North York are 43.7615, -79.4111.


In [30]:
map_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ny)  
    
map_ny

In [31]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Hillcrest Village" are 43.8037622, -79.3634517.


In [32]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny, VERSION, radius, LIMIT)

In [33]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 3.Etobiocke and its neighbourhoods

In [36]:
etobicoke_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
5,Etobicoke,Islington Avenue,43.667856,-79.532242
6,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724


In [37]:
etobicoke_data.shape

(12, 4)

In [38]:
address_et = 'Etobicoke,Toronto'
latitude_et = 43.6205
longitude_et = -79.5132
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude_et, longitude_et))

The geograpical coordinate of Etobicoke are 43.6205, -79.5132.


In [39]:
map_et = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_et)  
    
map_et

In [40]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Humber Bay Shores, Mimico South, New Toronto" are 43.6056466, -79.50132070000001.


In [41]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_et, longitude_et, VERSION, radius, LIMIT)

In [42]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [43]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 4.East York and its neighbourhoods

In [45]:
eastyork_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'East York'].reset_index(drop=True)
eastyork_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
1,East York,Woodbine Heights,43.695344,-79.318389
2,East York,Leaside,43.709060,-79.363452
3,East York,Thorncliffe Park,43.705369,-79.349372
4,East York,East Toronto,43.685347,-79.338106


In [46]:
eastyork_data.shape

(5, 4)

In [47]:
address_ey = 'East York,Toronto'
latitude_ey = 43.6912
longitude_ey = -79.3417
print('The geograpical coordinate of East York are {}, {}.'.format(latitude_ey, longitude_ey))

The geograpical coordinate of East York are 43.6912, -79.3417.


In [48]:
map_ey = folium.Map(location=[latitude_ey, longitude_ey], zoom_start=12)

# add markers to map
for lat, lng, label in zip(eastyork_data['Latitude'], eastyork_data['Longitude'], eastyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ey)  
    
map_ey

In [49]:
neighborhood_latitude = eastyork_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = eastyork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = eastyork_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Woodbine Gardens, Parkview Hill" are 43.7063972, -79.309937.


In [50]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ey, longitude_ey, VERSION, radius, LIMIT)

In [51]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [52]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 5. Downtown Toronto and its neighbourhoods

In [54]:
downtown_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Rosedale,43.679563,-79.377529
1,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,Downtown Toronto,St. James Town,43.651494,-79.375418
6,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [55]:
downtown_data.shape

(18, 4)

In [56]:
address_dt = 'Downtown Toronto,Toronto'
latitude_dt = 43.6543
longitude_dt = -79.3860
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_dt, longitude_dt))

The geograpical coordinate of Downtown Toronto are 43.6543, -79.386.


In [57]:
map_dt = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dt)  
    
map_dt

In [58]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rosedale" are 43.6795626, -79.37752940000001.


In [59]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_dt, longitude_dt, VERSION, radius, LIMIT)

In [60]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [61]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 6. York and its neighborhoods

In [63]:
york_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'York'].reset_index(drop=True)
york_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,York,Humewood-Cedarvale,43.693781,-79.428191
1,York,Caledonia-Fairbanks,43.689026,-79.453512
2,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
3,York,"The Junction North, Runnymede",43.673185,-79.487262
4,York,Weston,43.706876,-79.518188


In [64]:
address_y = 'York,Toronto'
latitude_y = 43.6543
longitude_y = -79.3860
print('The geograpical coordinate of York are {}, {}.'.format(latitude_y, longitude_y))

The geograpical coordinate of York are 43.6543, -79.386.


In [65]:
york_data.shape

(5, 4)

In [66]:
map_york = folium.Map(location=[latitude_y, longitude_y], zoom_start=12)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_york)  
    
map_york

In [67]:
neighborhood_latitude = york_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = york_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = york_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Humewood-Cedarvale" are 43.6937813, -79.42819140000002.


In [68]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_y, longitude_y, VERSION, radius, LIMIT)

In [69]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [70]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 7. East Toronto and its neighborhood

In [72]:
easttoronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'East Toronto'].reset_index(drop=True)
easttoronto_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [73]:
address_et = 'East Toronto,Toronto'
latitude_et = 43.6780
longitude_et = -79.3099
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude_et, longitude_et))

The geograpical coordinate of East Toronto are 43.678, -79.3099.


In [74]:
easttoronto_data.shape

(5, 4)

In [75]:
map_et = folium.Map(location=[latitude_et, longitude_et], zoom_start=12)

# add markers to map
for lat, lng, label in zip(easttoronto_data['Latitude'], easttoronto_data['Longitude'], easttoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_et)  
    
map_et

In [76]:
neighborhood_latitude = easttoronto_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = easttoronto_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = easttoronto_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "The Beaches" are 43.67635739999999, -79.2930312.


In [77]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_y, longitude_y, VERSION, radius, LIMIT)

In [78]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 8. Central Toronto and its neighborhoods

In [81]:
centraltoronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
centraltoronto_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,North Toronto West,43.715383,-79.405678
3,Central Toronto,Davisville,43.704324,-79.388790
4,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
6,Central Toronto,Roselawn,43.711695,-79.416936


In [82]:
address_ct = 'Central Toronto,Toronto'
latitude_ct = 43.6577
longitude_ct = -79.4216
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude_ct, longitude_ct))

The geograpical coordinate of Central Toronto are 43.6577, -79.4216.


In [83]:
centraltoronto_data.shape

(9, 4)

In [84]:
map_ct= folium.Map(location=[latitude_ct, longitude_ct], zoom_start=12)

# add markers to map
for lat, lng, label in zip(centraltoronto_data['Latitude'], centraltoronto_data['Longitude'], centraltoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ct)  
    
map_ct

In [85]:
neighborhood_latitude = centraltoronto_data.loc[2, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = centraltoronto_data.loc[2, 'Longitude'] # neighbourhood longitude value

neighborhood_name = centraltoronto_data.loc[2, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "North Toronto West" are 43.7153834, -79.40567840000001.


In [86]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ct, longitude_ct, VERSION, radius, LIMIT)

In [87]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [88]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [89]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

In [90]:
#SGnearby_venues

# 9. Queen's Park and its neighborhoods

In [91]:
queenspark_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == "Queen's Park"].reset_index(drop=True)
queenspark_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Queen's Park,Queen's Park,43.662301,-79.389494


In [92]:
address_qp = 'Queens Park,Toronto'
latitude_qp = 43.6644
longitude_qp = -79.3923
print('The geograpical coordinate of Queens Park are {}, {}.'.format(latitude_qp, longitude_qp))

The geograpical coordinate of Queens Park are 43.6644, -79.3923.


In [93]:
queenspark_data.shape

(1, 4)

In [94]:
map_qp = folium.Map(location=[latitude_qp, longitude_qp], zoom_start=12)

# add markers to map
for lat, lng, label in zip(queenspark_data['Latitude'], queenspark_data['Longitude'], queenspark_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_qp)  
    
map_qp

In [95]:
neighborhood_latitude = queenspark_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = queenspark_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = queenspark_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Queen's Park" are 43.6623015, -79.3894938.


In [96]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_qp, longitude_qp, VERSION, radius, LIMIT)

In [97]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [98]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

In [100]:
#SGnearby_venues.head(44)

# 10. Mississauga and its neighborhoods

In [101]:
mississauga_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Mississauga'].reset_index(drop=True)
mississauga_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819


In [102]:
mississauga_data.shape

(1, 4)

In [103]:
address_m = 'Mississauga,Toronto'
latitude_m = 43.5890
longitude_m = -79.6441
print('The geograpical coordinate of Mississauga are {}, {}.'.format(latitude_m, longitude_m))

The geograpical coordinate of Mississauga are 43.589, -79.6441.


In [104]:
mississauga_data.shape

(1, 4)

In [105]:
map_m = folium.Map(location=[latitude_m, longitude_m], zoom_start=12)

# add markers to map
for lat, lng, label in zip(mississauga_data['Latitude'], mississauga_data['Longitude'], mississauga_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_m)  
    
map_m

In [106]:
neighborhood_latitude = mississauga_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = mississauga_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = mississauga_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Canada Post Gateway Processing Centre" are 43.6369656, -79.61581899999999.


In [107]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_et, longitude_et, VERSION, radius, LIMIT)

In [108]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [109]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [110]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(5, 4)

In [111]:
SGnearby_venues.head(44)

,name,categories,lat,lng
0,Budapest Restaurant,Hungarian Restaurant,43.680946,-79.310110
1,Bodega Henriette,Café,43.678077,-79.314868
2,East Of Eliza,Flower Shop,43.681662,-79.310512
3,Cassels Park,Park,43.681725,-79.310489
4,Sandy's Restaurant,Asian Restaurant,43.681766,-79.310512


# 11. West Toronto and its neighborhood

In [112]:
westtoronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'West Toronto'].reset_index(drop=True)
westtoronto_data.head(7)

,Borough,Neighborhood,Latitude,Longitude
0,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


In [113]:
address_wt = 'West Toronto,Toronto'
latitude_wt = 43.6534
longitude_wt = -79.4259
print('The geograpical coordinate of West Toronto are {}, {}.'.format(latitude_wt, longitude_wt))

The geograpical coordinate of West Toronto are 43.6534, -79.4259.


In [114]:
westtoronto_data.shape

(6, 4)

In [115]:
map_wt = folium.Map(location=[latitude_wt, longitude_wt], zoom_start=12)

# add markers to map
for lat, lng, label in zip(westtoronto_data['Latitude'], westtoronto_data['Longitude'], westtoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_wt)  
    
map_wt

In [116]:
neighborhood_latitude = westtoronto_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = westtoronto_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = westtoronto_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Dovercourt Village, Dufferin" are 43.66900510000001, -79.4422593.


In [117]:
LIMIT = 10
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_y, longitude_y, VERSION, radius, LIMIT)

In [118]:
results = requests.get(url).json()
#results

function that extracts the category of the venue - borrow from the Foursquare lab.

In [119]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [120]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(10, 4)

# 2. Explore Neighborhoods in Toronto

#### A function to repeat the same process to all the neighborhoods in Toronto

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### The code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [122]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [123]:
print(toronto_venues.shape)
toronto_venues.head()

(983, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant


## how many venues were returned for each neighborhood?

In [124]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",10,10,10,10,10,10
Agincourt,10,10,10,10,10,10
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",10,10,10,10,10,10
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",10,10,10,10,10,10
Bayview Village,10,10,10,10,10,10
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10


#### How many unique categories can be curated from all the returned venues?

In [125]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 182 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [126]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Museum,Music Store,Neighborhood,New American Restaurant,Noodle House,Office,Organic Grocery,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [127]:
toronto_onehot.shape

(983, 182)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [128]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Noodle House,Office,Organic Grocery,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Ski Chalet,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.1,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.000,0.000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.1,0.000,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.1,0.2,0.0,0.100,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.100,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0

#### The new size

In [129]:
toronto_grouped.shape

(102, 182)

#### Print each neighborhood along with the top 5 most common venues

In [130]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0                          Hotel   0.1
1             Seafood Restaurant   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                          Plaza   0.1
4                   Concert Hall   0.1


----Agincourt----
                  venue  freq
0  Caribbean Restaurant   0.2
1          Noodle House   0.1
2  Cantonese Restaurant   0.1
3     Indian Restaurant   0.1
4        Breakfast Spot   0.1


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                           venue  freq
0                    Coffee Shop   0.1
1              Korean Restaurant   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                   Dessert Shop   0.1
4           Caribbean Restaurant   0.1


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0          Pizza Place   0.3
1        Grocery Store   0.3
2             Pharmacy   

#### Put that into a *pandas* dataframe

A function to sort the venues in descending order.

In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [132]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted=[]
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Concert Hall,Food Court,Seafood Restaurant,Vegetarian / Vegan Restaurant,Greek Restaurant,Speakeasy,Plaza,Hotel,Steakhouse,Empanada Restaurant
1,Agincourt,Caribbean Restaurant,Supermarket,Breakfast Spot,Noodle House,Sri Lankan Restaurant,Indian Restaurant,Cantonese Restaurant,Bakery,Chinese Restaurant,Wings Joint
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Bakery,Coffee Shop,Korean Restaurant,Event Space,Caribbean Restaurant,Chinese Restaurant,Dessert Shop,Malay Restaurant,Vegetarian / Vegan Restaurant,Noodle House
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Dessert Shop,Event Space,Empanada Restaurant,Electronics Store
4,"Alderwood, Long Branch",Grocery Store,Pizza Place,Gym,Coffee Shop,Discount Store,Pub,Moroccan Restaurant,Sandwich Place,Wings Joint,Dessert Shop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [133]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:102] 

array([1, 1, 8, 5, 5, 2, 8, 3, 1, 0, 5, 3, 1, 3, 2, 0, 5, 1, 1, 3, 0, 3,
       1, 2, 8, 1, 5, 0, 0, 3, 3, 9, 0, 0, 2, 3, 4, 5, 2, 2, 2, 0, 1, 1,
       0, 8, 3, 2, 5, 8, 1, 2, 8, 3, 5, 9, 3, 5, 3, 5, 5, 8, 5, 1, 0, 1,
       8, 9, 3, 9, 3, 7, 2, 8, 5, 3, 2, 3, 2, 3, 0, 2, 6, 2, 1, 0, 1, 1,
       1, 8, 1, 8, 1, 3, 5, 5, 0, 5, 2, 5, 1, 9], dtype=int32)

In [134]:
toronto_grouped_clustering.shape

(102, 181)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Spa,African Restaurant,Hobby Shop,Paper / Office Supplies Store,Coffee Shop,Martial Arts Dojo,Caribbean Restaurant,Dog Run,Farmers Market
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,Breakfast Spot,Playground,Park,Burger Joint,Italian Restaurant,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5.0,Pizza Place,Pharmacy,Coffee Shop,Liquor Store,Beer Store,Food & Drink Shop,Discount Store,Fried Chicken Joint,Smoothie Shop,Dance Studio
M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Pharmacy,Park,Electronics Store,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Discount Store,Farmers Market,Falafel Restaurant
M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Indian Restaurant,Coffee Shop,Athletics & Sports,Caribbean Restaurant,Chinese Restaurant,Thai Restaurant,Music Store,Burger Joint,Hakka Restaurant,Dessert Shop


In [136]:
toronto_merged.shape

(103, 15)

Finally, let's visualize the resulting clusters

In [137]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


TypeError: list indices must be integers or slices, not float

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 6

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 7

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 8

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 9

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 10

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]